# Phase Diagram for the Syntectic System

Here we develop a free energy functional and phase diagram for a system with a syntectic transition

## Ideal Free Energy

---

The ideal free energy per unit area is of the following form

$$
\mathcal{F}_{id}[n] = \frac{1}{V_{cell}}\int_{cell} 
\frac{1}{2}n(x,y)^2 - \frac{\eta}{6} n(x,y)^3 + \frac{\chi}{12} n(x,y)^4
$$

To compute the phase diagram we use a variational method where we expand the reduced density, $n$, in the most general fourier series that is consistent with the symmetry of the bravais lattice.

$$
n(x,y) = n_0 + A_1 n_1(x,y) + A_2 n_2(x,y) + ...
$$

We then expand out the ansatz and compute each integral over the unit-cell numerically to find the amplitude equation for the ideal free energy

In [1]:
using PyPlot
using Interact
using Maxima
using Optim
using PhaseDiagrams

In [2]:
f_ideal = m"n^2/2 - η*n^3/6 + χ*n^4/12"

 
                                4      3      2
                               n  χ   n  η   n
                               ---- - ---- + --
                                12     6     2

In [3]:
n = m"n0 + 2*A*n1"            # + A[2]*n2"

f_ideal = subst(n, :n, f_ideal)

 
                          4                  3                  2
             (2 A n1 + n0)  χ   (2 A n1 + n0)  η   (2 A n1 + n0)
             ---------------- - ---------------- + --------------
                    12                 6                 2

In [4]:
f_ideal = expand(f_ideal)

 
    4   4        3      3                            3          4
 4 A  n1  χ   8 A  n0 n1  χ      2   2   2     2 A n0  n1 χ   n0  χ
 ---------- + ------------- + 2 A  n0  n1  χ + ------------ + -----
     3              3                               3          12
      3   3                                    3                              2
   4 A  n1  η      2      2         2        n0  η      2   2               n0
 - ---------- - 2 A  n0 n1  η - A n0  n1 η - ----- + 2 A  n1  + 2 A n0 n1 + ---
       3                                       6                             2

In [5]:
n1 = m"cos(4*%pi/sqrt(3)*(sqrt(3)*x/2 - y/2)) + 
       cos(4*%pi/sqrt(3)*(sqrt(3)*x/2 + y/2)) + 
       cos(4*%pi/sqrt(3)*y)"

n2 = m"cos(4*%pi*x) +
       cos(4*%pi*(x/2 + sqrt(3)*y/2)) + 
       cos(4*%pi*(-x/2 + sqrt(3)*y/2))"

f_ideal = subst(n1, :n1, f_ideal)
#f_ideal = subst(n2, :n2, f_ideal);

In [6]:
F_ideal = "integrate(integrate($f_ideal, x, 0, 1), y, 0, sqrt(3)/2)" |> MExpr;

In [7]:
F_ideal = mcall(F_ideal)

 
            3/2       4       7/2      2   2       5/2      3
 ((- ((- 2 3    %pi n0 ) - 8 3    %pi A  n0  - 32 3    %pi A  n0
       7/2      4           3/2       3      7/2      2          5/2      3
 - 20 3    %pi A ) χ) - (4 3    %pi n0  + 8 3    %pi A  n0 + 16 3    %pi A ) η
      5/2       2      7/2      2
 + 4 3    %pi n0  + 8 3    %pi A )/(144 %pi)

In [8]:
F_ideal = ratsimp(F_ideal)
F_ideal = "$F_ideal/(sqrt(3)/2)" |> MExpr |> mcall
F_ideal = ratsimp(F_ideal)
F_ideal = float(F_ideal)
F_ideal = expand(F_ideal)

 
                       4          2   2          3             4
 0.08333333333333333 n0  χ + 3.0 A  n0  χ + 4.0 A  n0 χ + 7.5 A  χ
                                3          2             3           2        2
         - 0.1666666666666667 n0  η - 3.0 A  n0 η - 2.0 A  η + 0.5 n0  + 3.0 A

In [9]:
F_id_exp = parse(F_ideal)                # Make julia expression of the ideal free energy
@eval F_id(η, χ, n0, A) = $F_id_exp     # Make julia function from the expression

F_id (generic function with 1 method)

## Free Energy of Mixing

The free energy of mixing is of the following form:

$$
    \mathcal{F}_{mix}[c] = \frac{1}{V_{cell}}\int_{cell} 
    (1 + n_0)\left(c \log\left(\frac{c}{c_0}\right) + 
    (1-c) \log\left(\frac{1-c}{1-c_0}\right)\right) + \frac{\epsilon(T)}{2}(c-c_0)^2
$$

In equilibrium we can assume that the concentration is constant so the integral just cancels the factor of the volume of the unit cell. The term $\epsilon(T)$ is the enthaply of mixing term which we model as, 

$$
\epsilon(T) = -4 + \epsilon_0 (T - T_0),
$$

where $T_0$ is the spinodal temperature of the liquid

In [10]:
 F_mix(c, T, n₀, ϵ₀, Tc, ω) = ω*((1+n₀)*(c*log(2.0*c) + (1-c)*log(2.0*(1-c))) + (-4.0 + ϵ₀*(T-Tc))/2*(c-0.5)^2)

F_mix (generic function with 1 method)

## Correlation functions

There are a variety of effective pair correlation functions we can choose from. Here is one of them from the original xpfc paper

In [11]:
const T₀ = 1.0
const k′ = 2π
const α = 0.8
#β = 6.0
#ρ = sqrt(3)/2.0
const αc = 0.5
const c₀ = 0.5

C_eff(k, c, T) = exp(-(c-c₀)^2/(2*(αc)^2))*exp(-T/T₀)*exp(-(k-k′)^2/(2*α^2))

C_eff (generic function with 1 method)

## Excess Free Energy

The excess free energy term in the binary phase field crystal model is the form

$$
\mathcal{F}_{ex}[n, c] = -\frac{1}{2 V_{cell}}\int_{cell} dr \int dr^\prime n(r) C_{eff}(r, r^\prime; c) n(r^\prime)
$$

Most of the modes in the amplitude expansion cancel out but we're left with the following at the end of the day:

$$
\mathcal{F}_{ex}[n, c] = -\frac{1}{2 V_{cell}}(n_0^2 C_{eff}(k=0, c) + 6 A_1^2 C_{eff}(k=k_1, c) + 6 A_2^2 C_{eff}(k=k_2, c))
$$

In [12]:
function F_ex(c, T, A)
    return (-3.0*A^2*C_eff(2π, c, T))       # + 6.0*A[2]^2*C_eff(k[2], c, T))
end

F_ex (generic function with 1 method)

## Total free energy

The total free energy is sum of all the above terms

In [13]:
F(c, T, η, χ, ϵ₀, T₀, ω, n₀, A) = F_mix(c, T, n₀, ϵ₀, T₀, ω) + F_ex(c, T, A)+ F_id(η, χ, n₀, A) 

F (generic function with 1 method)

In [14]:
function F(c, T, η, χ, ϵ₀, T₀, ω, n₀)
    sol = optimize(A -> F(c, T, η, χ, ϵ₀, T₀, ω, n₀, A), 0.0, 10.0)
    #println("Amplitude at concentation $c and temperature $T is $(sol.minimum)")
    return sol.minimum
end

F (generic function with 2 methods)

In [15]:
const η = 2.0
const χ = 1.0
const ϵ₀ = 10.0
const Tc = 0.35
const ω = 0.3
const n₀ = 0.05

F(c, T) = F(c, T, η, χ, ϵ₀, Tc, ω, n₀)

F (generic function with 3 methods)

In [34]:
function make_phase_diagram(c, T)
    T_vals = Float64[] 
    c_vals = Float64[]
    
    f = Array{Float64}(length(c))
    
    for t in 1:length(T)
        for cc in 1:length(c)
            f[cc] = F(c[cc], T[t])
        end
        arr = commontangent(f)
        for domain in arr
            for point in domain
                push!(T_vals, T[t])
                push!(c_vals, c[point])
            end
        end
    end
    return T_vals, c_vals
end

make_phase_diagram (generic function with 1 method)

In [63]:
crnge = collect(0.001:0.0005:0.999)
Trnge = collect(0.00:0.005:0.35)

fine = collect(0.325:0.0015:0.34)
push!(Trnge, fine...)

T_vals, c_vals = make_phase_diagram(crnge, Trnge)

([0.0, 0.0, 0.0, 0.0, 0.005, 0.005, 0.005, 0.005, 0.01, 0.01  …  0.325, 0.325, 0.325, 0.325, 0.3265, 0.3265, 0.328, 0.328, 0.3295, 0.3295], [0.0275, 0.447, 0.553, 0.9725, 0.0285, 0.448, 0.552, 0.9715, 0.0295, 0.4495  …  0.4065, 0.5935, 0.4065, 0.5935, 0.4215, 0.5785, 0.4405, 0.5595, 0.47, 0.53])

In [67]:
f = figure()
withfig(f) do
    scatter(c_vals, T_vals, c="k", s=2)
    grid(false)
    xlim(0, 1)
    ylim(0.00, 0.40)
    xlabel(L"Concentration $(c)$", fontsize=14)
    ylabel(L"Temperature $(T)$", fontsize=14)
end

savefig("Syntectic Phase Diagram.svg")

In [ ]:
scatter(c_vals, T_vals, c="k", s=10)
xlim(0, 1)
ylim(0.0, 0.40)
xlabel(L"Concentration $(c)$", fontsize=14)
ylabel(L"Temperature $(T)$", fontsize=14)
grid(false)